In [1]:
#IMPORTANDO BIBLIOTECAS
import requests
import json
import pandas as pd
import sqlite3
import time
from tqdm import tqdm
import datetime
import os

In [2]:
#GUARDANDO LISTAS
Urls_Magalu = []
Sellers_Magalu = []
Country_Magalu = []
Price_Magalu = []
SKU_Magalu = []
Title_Magalu = []
Installment_Magalu_quantidade = []
Installment_Magalu_valor_parcela = []
Installment_Magalu_valor_total = []

In [3]:
import pymysql

In [4]:
def getting_n_creating_magazine_urls(brand):
    connection = pymysql.connect(host='mysqlserver.cnzboqhfvndh.sa-east-1.rds.amazonaws.com',
                                 user='admin',
                                 password='turtle316712',
                                 database='Products_Brands',
                                 cursorclass=pymysql.cursors.DictCursor)

    #Criando o caminho do Databae
    c = connection.cursor()

    #Criando a Query
    Sql_query = "SELECT * FROM Products WHERE Brand = '%s'" % (brand)

    #Conectando com o banco de dados
    c.execute(Sql_query)
    result = c.fetchall()
    c.close()
    connection.close()

    #Passando todos o dataframe para Lowercase
    df = pd.DataFrame()
    df['Brand'] = [item['Brand'] for item in result]
    df['Product_Name'] = [item['Name'] for item in result]

    # Criando uma nova coluna no database com a formatação certa
    df['Urls'] = df['Brand'] + "+" + df['Product_Name']

    # Criando a nova coluna que são as urls de pesquisa
    df['Urls_search'] = "https://www.magazineluiza.com.br/_next/data/ZIpIaOx0q3rMCfEgx3tvz/busca/" + \
    df['Brand'][0] + "%2B" + df['Product_Name'] + ".json?slug=busca&slug=" + \
    df['Brand'][0] + "%2B" + df['Product_Name']

    return df

In [5]:
Df_products_raw = getting_n_creating_magazine_urls("Wacom")
Df_products_raw

,Brand,Product_Name,Urls,Urls_search
0,Wacom,CTL472,Wacom+CTL472,https://www.magazineluiza.com.br/_next/data/ZI...
1,Wacom,CTL4100,Wacom+CTL4100,https://www.magazineluiza.com.br/_next/data/ZI...
2,Wacom,CTL4100WLE0,Wacom+CTL4100WLE0,https://www.magazineluiza.com.br/_next/data/ZI...
3,Wacom,CTL4100WLK0,Wacom+CTL4100WLK0,https://www.magazineluiza.com.br/_next/data/ZI...
4,Wacom,CTL6100WLK0,Wacom+CTL6100WLK0,https://www.magazineluiza.com.br/_next/data/ZI...
5,Wacom,DTC133W0A,Wacom+DTC133W0A,https://www.magazineluiza.com.br/_next/data/ZI...
6,Wacom,PTH460,Wacom+PTH460,https://www.magazineluiza.com.br/_next/data/ZI...
7,Wacom,DTK1660K0A,Wacom+DTK1660K0A,https://www.magazineluiza.com.br/_next/data/ZI...
8,Wacom,PTH660,Wacom+PTH660,https://www.magazineluiza.com.br/_next/data/ZI...
9,Wacom,CTL6100WLE0,Wacom+CTL6100WLE0,https://www.magazineluiza.com.br/_next/data/ZI...


In [33]:
Df_products_raw['Urls_search'][0]

'https://www.magazineluiza.com.br/_next/data/ZIpIaOx0q3rMCfEgx3tvz/busca/Boya%2Bmc2.json?slug=busca&slug=Boya%2Bmc2'

In [5]:
urls = 'https://www.magazineluiza.com.br/_next/data/ZIpIaOx0q3rMCfEgx3tvz/busca/toshiba%2Bhd%2Bexterno%2Bcanvio%2Bbasics%2B1tb.json?path1=toshiba%2Bhd%2Bexterno%2Bcanvio%2Bbasics%2B1tb','https://www.magazineluiza.com.br/_next/data/ZIpIaOx0q3rMCfEgx3tvz/busca/hd%2Bexterno%2Bseagate%2Bexpansion%2B1tb.json?path1=hd%2Bexterno%2Bseagate%2Bexpansion%2B1tb'

In [ ]:
Toshiba HD externo Canvio Basics
HD Externo Seagate Expansion

In [6]:
headers_magalu = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 OPR/86.0.4363.59"}

In [7]:
response = requests.get(url, headers=headers_magalu)
texto_json = response.json()

InvalidSchema: No connection adapters were found for "('https://www.magazineluiza.com.br/_next/data/ZIpIaOx0q3rMCfEgx3tvz/busca/Godox%2Bsk300.json?slug=busca&slug=Godox%2Bsk300', 'https://www.magazineluiza.com.br/_next/data/ZIpIaOx0q3rMCfEgx3tvz/busca/YN600EXRT.json?slug=busca&slug=YN600EXRT')"

In [ ]:
texto_json['pageProps']['data']['search']['products']

In [7]:
def get_attributes(url):
    time.sleep(20)

    response = requests.get(url, headers=headers_magalu)
    text_json = response.json()

    # URL
    id_url = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            Urls_Magalu.append(text_json['pageProps']['data']['search']['products'][id_url]['url'])
        except:
            Urls_Magalu.append("ERRO")

        id_url = id_url + 1

    # ID
    id_number = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            SKU_Magalu.append(text_json['pageProps']['data']['search']['products'][id_number]['variationId'])
        except:
            SKU_Magalu.append("ERRO")

        id_number = id_number + 1

    # SELLER
    id_seller = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            Sellers_Magalu.append(
                text_json['pageProps']['data']['search']['products'][id_seller]['seller']['description'])
        except:
            Sellers_Magalu.append("ERRO")

        id_seller = id_seller + 1

    # COUNTRY
    id_country = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            Country_Magalu.append(
                text_json['pageProps']['data']['search']['products'][id_country]['seller']['details']['address'][
                    'country'])
        except:
            Country_Magalu.append("ERRO")

        id_country = id_country + 1

    # PREÇO
    id_price = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            Price_Magalu.append(text_json['pageProps']['data']['search']['products'][id_price]['price']['bestPrice'])
        except:
            Price_Magalu.append("ERRO")

        id_price = id_price + 1

    # TITULO
    id_title = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            Title_Magalu.append(text_json['pageProps']['data']['search']['products'][id_title]['title'])
        except:
            Title_Magalu.append("ERRO")

        id_title = id_title + 1

    # INSTALLMENT QUANTIDADE
    id_installment_quantidade = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            Installment_Magalu_quantidade.append(
                text_json['pageProps']['data']['search']['products'][id_installment_quantidade]['installment'][
                    'quantity'])
        except:
            Installment_Magalu_quantidade.append("ERRO")

        id_installment_quantidade = id_installment_quantidade + 1

    # INSTALLMENT VALOR POR PARCELA
    id_installment_valor_parcela = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            Installment_Magalu_valor_parcela.append(
                text_json['pageProps']['data']['search']['products'][id_installment_valor_parcela]['installment'][
                    'amount'])
        except:
            Installment_Magalu_valor_parcela.append("ERRO")

        id_installment_valor_parcela = id_installment_valor_parcela + 1

    # INSTALLMENT VALOR TOTAL
    id_installment_valor_total = 0
    for i in text_json['pageProps']['data']['search']['products']:
        try:
            Installment_Magalu_valor_total.append(
                text_json['pageProps']['data']['search']['products'][id_installment_valor_total]['installment'][
                    'totalAmount'])
        except:
            Installment_Magalu_valor_total.append("ERRO")

        id_installment_valor_total = id_installment_valor_total + 1

In [8]:
for url in tqdm(Df_products_raw['Urls_search']):
    get_attributes(url)

100%|██████████| 10/10 [03:33<00:00, 21.32s/it]


In [43]:
get_attributes(url)

In [34]:
Urls = ['https://www.magazineluiza.com.br/_next/data/cpGkZe7hwv0NoZfmUVN-U/busca/dji%2Bmavic%2Bmini%2B2%2Bfly%2Bcombo.json?slug=busca&slug=dji%2Bmavic%2Bmini%2B2%2Bfly%2Bcombo',
        'https://www.magazineluiza.com.br/_next/data/cpGkZe7hwv0NoZfmUVN-U/busca/dji%2Bmini%2Bse%2Bfly%2Bmore%2Bcombo.json?slug=busca&slug=dji%2Bmini%2Bse%2Bfly%2Bmore%2Bcombo']

In [35]:
for url in tqdm(Urls):
    get_attributes(url)

100%|██████████| 2/2 [00:41<00:00, 20.72s/it]


In [9]:
def creating_dataframe(urls, sellers, price, installment, parcel, installment_total, sku, title, brand):
    Dataframe = pd.DataFrame()

    Hoje = pd.to_datetime('today', errors='ignore').date()

    Dataframe['Urls_2'] = urls

    Dataframe['DATE'] = Hoje

    Dataframe['URL'] = "https://www.magazineluiza.com.br/" + Dataframe['Urls_2']
    Dataframe['MARKETPLACE'] = 'MAGAZINE LUIZA'
    Dataframe['SELLER'] = sellers

    Dataframe['PRICE'] = price
    Dataframe['PRICE'] = Dataframe['PRICE'].astype('float')


    Dataframe['INSTALLMENT'] = installment

    Dataframe['PARCEL'] = parcel


    Dataframe['INSTALLMENT_PAYMENT'] = installment_total
    #Dataframe['INSTALLMENT_PAYMENT'] = Dataframe['INSTALLMENT_PAYMENT'].astype('float')

    Dataframe['ID'] = sku
    Dataframe['PRODUCT'] = title

    if brand == "GoPro":
        Dataframe = Dataframe[Dataframe['PRICE'] > 1000]
    elif brand == 'Motorola':
        Dataframe = Dataframe[Dataframe['PRICE'] > 100]
    elif brand == 'Wacom':
        Dataframe = Dataframe[Dataframe['PRICE'] > 100]

    #Colocando na ordem correta
    Dataframe = Dataframe[['DATE', 'URL', 'MARKETPLACE', 'SELLER', 'PRICE', 'PARCEL', 'INSTALLMENT', 'INSTALLMENT_PAYMENT', 'ID', 'PRODUCT']]

    Dataframe['INSTALLMENT'] = Dataframe['INSTALLMENT'].astype('float')
    Dataframe['PARCEL'] = Dataframe['PARCEL'].astype('int')
    Dataframe['INSTALLMENT_PAYMENT'] = Dataframe['INSTALLMENT_PAYMENT'].astype('float')

    return Dataframe

In [10]:
dataset_magalu = creating_dataframe(Urls_Magalu, Sellers_Magalu, Price_Magalu, Installment_Magalu_valor_parcela, Installment_Magalu_quantidade, Installment_Magalu_valor_total, SKU_Magalu, Title_Magalu, 'Wacom')

In [11]:
dataset_magalu

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT
0,2022-08-01,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,BRDF Informática,489.00,9,54.33,489.00,gda617467j,Mesa Digitalizadora Wacom CTL472
1,2022-08-01,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,Move Tech,357.93,6,61.50,369.00,hg2dj8ja7b,Mesa Digitalizadora Wacom Pequena - One CTL472
2,2022-08-01,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,Info Store,387.95,7,57.14,399.95,gk38jg1077,Mesa Digitalizadora One By Wacom CTL472
3,2022-08-01,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,Universal Informática 5,359.00,6,59.83,359.00,efjk7g95dg,Mesa Digitalizadora One by Wacom CTL472 Pequen...
4,2022-08-01,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,Casa Da Informática Phb,499.90,9,55.54,499.90,fbg576ek2e,Mesa Digitalizadora Wacom Pequena - One CTL472
...,...,...,...,...,...,...,...,...,...,...
92,2022-08-01,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,W-Tech Imports,3531.21,10,379.70,3797.00,ba17h1c372,Mesa Digitalizadora Wacom Intuos Pro Media PTH660
93,2022-08-01,https://www.magazineluiza.com.br/caneta-wacom-...,MAGAZINE LUIZA,e-spot,949.90,10,94.99,949.90,gfak2efjc9,Caneta Wacom Ballpoint p/ Intuos Pro (PTH660/8...
94,2022-08-01,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,e-spot,1659.90,10,165.99,1659.90,khg7205477,Mesa Digitalizadora Wacom Intuos Bluetooth Méd...
95,2022-08-01,https://www.magazineluiza.com.br/mesa-digitali...,MAGAZINE LUIZA,Inpower,1475.10,10,163.90,1639.00,eah018g7h7,Mesa Digitalizadora Wacom Intuos M Bluetooth P...


In [12]:
dataset_magalu.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Software_Artemis\Data\Brands_Downloads\Wacom\Magazine_Wacom.xlsx", index=False)

In [58]:
dataset_magalu[dataset_magalu['INSTALLMENT'] == 'ERRO']

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT


In [71]:
dataset_magalu.dtypes

DATE                    object
URL                     object
MARKETPLACE             object
SELLER                  object
PRICE                  float64
PARCEL                   int32
INSTALLMENT            float64
INSTALLMENT_PAYMENT    float64
ID                      object
PRODUCT                 object
dtype: object

In [40]:
current = os.getcwd()
current

'C:\\Users\\pedro\\Documents\\Turte Brand Protection\\Turtle_Thinker_Alpha_0.1\\Testes_Spiders'

In [41]:
path = current + '\Teste_Magalu.xlsx'

In [21]:
path

'c:\\Users\\pedro\\Documents\\Turte Brand Protection\\Turtle_Thinker_Alpha_0.1\\Testes_Spiders\\Teste_Magalu.xlsx'

In [73]:
dataset_magalu.to_excel(path, index=False)

In [16]:
dataset_magalu.dtypes

DATE                    object
URL                     object
MARKETPLACE             object
SELLER                  object
PRICE                  float64
PARCEL                  object
INSTALLMENT             object
INSTALLMENT_PAYMENT     object
ID                      object
PRODUCT                 object
dtype: object